In [1]:
import numpy as np
import os

import random
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import astropy.units as u
import matplotlib.ticker as mtick
import astropy.constants as const
import scipy.stats as stats
from matplotlib.lines import Line2D
import scipy.interpolate, scipy.optimize

In [2]:
import sys

from s1s2_0311 import *
from read import *
from recoil_1223 import *
from plot_binned_detection_1d2d_0526 import *

In [3]:
pcles_ER= ['pp','Be7_384', 'Be7_861', 'CNO', 'pep', 'nubb', 'Kr85', 'Rn222']
det_pcles = ['CNO', 'pep']

In [4]:
metallicities = ['high', 'low']
size = 100000
folder_1d = 'unbinned_Er_pdf_itp'

unit_pdf = (u.tonne*u.yr*u.keV)**(-1)
unit_Er = u.keV
eff = ''
corr = ''

In [6]:

binned_data_pd = pd.DataFrame()

#XENON IDEAL ER
binned_data_pd = get_1d_pd_ER(binned_data_pd, pcles_ER, [eff], 'Xenon', 
                              metallicities, bg_ideal ='ideal', binnum =  100, 
      folder_1d = '../unbinned_Er_pdf_itp', size = 100000, corr =corr, corr_sim = corr, 
         E_threshold_keV = 1, unit_pdf = (u.tonne*u.yr*u.keV)**(-1), unit_Er = u.keV, log = True, 
         print_check = False, plot = False)


ideal
Xenon ['pp', 'Be7_384', 'Be7_861', 'CNO', 'pep']
ER
else
 ../real_data_nest/pdf/pp_ER_Ebind_Xenon_pdf_high.txt
survival
other
ER
else
 ../real_data_nest/pdf/Be7_384_ER_Ebind_Xenon_pdf_high.txt
survival
other
ER
else
 ../real_data_nest/pdf/Be7_861_ER_Ebind_Xenon_pdf_high.txt
survival
other
ER
else
 ../real_data_nest/pdf/CNO_ER_Ebind_Xenon_pdf_high.txt
survival
other
ER
else
 ../real_data_nest/pdf/pep_ER_Ebind_Xenon_pdf_high.txt
survival
other
ER
else
 ../real_data_nest/pdf/pp_ER_Ebind_Xenon_pdf_low.txt
survival
other
ER
else
 ../real_data_nest/pdf/Be7_384_ER_Ebind_Xenon_pdf_low.txt
survival
other
ER
else
 ../real_data_nest/pdf/Be7_861_ER_Ebind_Xenon_pdf_low.txt
survival
other
ER
else
 ../real_data_nest/pdf/CNO_ER_Ebind_Xenon_pdf_low.txt
survival
other
ER
else
 ../real_data_nest/pdf/pep_ER_Ebind_Xenon_pdf_low.txt
survival
other


In [7]:
alphas_powers = [(0.001, 0.9), (0.01, 0.9), (0.1, 0.9)]
nubb_fraction = 0
methods = [ 'allMethod2', 'noppBe7Method2']

In [8]:
result_pd = pd.DataFrame(index = [str(alpha_power) for alpha_power in alphas_powers],
                         columns=[det_pcle+'_'+metallicity+'_nubb'+str(nubb_fraction)+'_'+method
                                  for det_pcle in det_pcles
                                  for  metallicity in metallicities
                                 for method in methods])


In [9]:
result_pd

,CNO_high_nubb0_allMethod2,CNO_high_nubb0_noppBe7Method2,CNO_low_nubb0_allMethod2,CNO_low_nubb0_noppBe7Method2,pep_high_nubb0_allMethod2,pep_high_nubb0_noppBe7Method2,pep_low_nubb0_allMethod2,pep_low_nubb0_noppBe7Method2
"(0.001, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.01, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.1, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
binned_data_pd.columns 

Index(['Xenon_Er_bin [keV]', 'Xenon_Er_bincenters [keV]',
       'high_pp_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'high_Be7_384_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'high_Be7_861_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'high_CNO_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'high_pep_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'low_pp_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'low_Be7_384_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'low_Be7_861_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'low_CNO_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)',
       'low_pep_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)'],
      dtype='object')

In [12]:
pcles_ppBe7 = ['pp','Be7_384']
exposures = np.logspace(np.log10(8), np.log10(500), 21)
Er_bincenters = list(binned_data_pd['Xenon_Er_bincenters [keV]'])

for det_pcle in det_pcles:
    
    for method in methods:
        if method == 'allMethod2':
            pcles_alter_ER, _ = get_all_components_ER(pcles_ER, 'Xenon', bg_ideal = 'ideal')
            pcles_null_ER = [n for n in pcles_alter_ER if n != det_pcle]

        
        elif method == 'noppBe7Method2':     
            pcles_alter_ER_all, _ = get_all_components_ER(pcles_ER, 'Xenon', bg_ideal = 'ideal')
            pcles_alter_ER = [p for p in pcles_alter_ER_all if p not in pcles_ppBe7]
            pcles_null_ER = [p for p in pcles_alter_ER if p not in [det_pcle]]
        print(det_pcle, method, '\npcles_null  = ', pcles_null_ER,
              '\npcles_alter = ', pcles_alter_ER)  
        
        
   
        for m, metallicity in enumerate(metallicities):
            select_col_null  = [col for col in binned_data_pd.columns if '_'+eff+'_' in col and corr in col and 
                                   metallicity in col and any(pcle in col for pcle in pcles_null_ER)]
            select_col_alter  = [col for col in binned_data_pd.columns if '_'+eff+'_' in col and corr in col and 
                                   metallicity in col and any(pcle in col for pcle in pcles_alter_ER)]

            select_col_ppBe7  = [col for col in binned_data_pd.columns 
                                if '_'+eff+'_' in col and ' '+corr+'1 / (t yr)' in col and 
                                metallicity in col and any(pcle in col for pcle in pcles_ppBe7)]
            print(select_col_ppBe7)

            if len(
                select_col_null) !=len(pcles_null_ER) or len(
                select_col_alter) !=len(pcles_alter_ER) or len(
                    select_col_ppBe7) !=len(pcles_ppBe7):
                print('wrong selection',
                     select_col_null, pcles_null_ER,'\n',
                     select_col_alter, pcles_alter_ER,'\n',
                         select_col_ppBe7, pcles_ppBe7)
                break
            else:

                rates_null = list(np.sum(binned_data_pd[select_col_null], axis = 1))/u.tonne/u.yr
                rates_alter = list(np.sum(binned_data_pd[select_col_alter], axis = 1))/u.tonne/u.yr
                rates_ppBe7 = list(np.sum(binned_data_pd[select_col_ppBe7], axis = 1))/u.tonne/u.yr
                
                if method == 'allMethod2':
                    null_sum_ijkappa = sum(rates_null).value
                    alter_sum_ijkappa = sum(rates_alter).value
                elif method == 'noppBe7Method2':          
                    good = rates_ppBe7==0
                    start_index, end_index = min(np.where(good)[0]), max(np.where(good)[0])
                    rates_null_good = rates_null[start_index:end_index+1]
                    rates_alter_good = rates_alter[start_index:end_index+1]
                    
                    null_sum_ijkappa = sum(rates_null_good).value
                    alter_sum_ijkappa = sum(rates_alter_good).value
                    
                    print('\nEr range keep noppBe7: ', len(rates_null_good), 'bins from ', 
                          Er_bincenters[start_index], ' to ', Er_bincenters[end_index], 
                          '\nnull_sum_ijkappa = ', null_sum_ijkappa, 
                          '\nalter_sum_ijkappa = ', alter_sum_ijkappa,

                      '\n')
                for alpha_power in alphas_powers:
                    alpha, power = alpha_power
                    DT,_,_,_,_ = get_nume_DT_alphabeta_G_method2(exposures, 
                                        null_sum_ijkappa, alter_sum_ijkappa, alpha, power, simnum = 500000, 
                                        plot= False, print_check = False)
                    result_pd[det_pcle+'_'+metallicity+'_nubb'+str(nubb_fraction)+'_'+method][str(alpha_power)] = DT

                    print(metallicity, nubb_fraction, 'alpha = ', alpha, 'power = ', power, 'DT = ',DT, '\n')



ideal
CNO allMethod2 
pcles_null  =  ['pp', 'Be7_384', 'Be7_861', 'pep'] 
pcles_alter =  ['pp', 'Be7_384', 'Be7_861', 'CNO', 'pep']
['high_pp_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)', 'high_Be7_384_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)']
high 0 alpha =  0.001 power =  0.9 DT =  41.61482122685396 

high 0 alpha =  0.01 power =  0.9 DT =  28.55279576425246 

high 0 alpha =  0.1 power =  0.9 DT =  14.408828540155817 

['low_pp_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)', 'low_Be7_384_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)']
low 0 alpha =  0.001 power =  0.9 DT =  80.54214604092134 

low 0 alpha =  0.01 power =  0.9 DT =  55.313346364071315 

low 0 alpha =  0.1 power =  0.9 DT =  27.55595717162864 

ideal
CNO noppBe7Method2 
pcles_null  =  ['Be7_861', 'pep'] 
pcles_alter =  ['Be7_861', 'CNO', 'pep']
['high_pp_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)', 'high_Be7_384_Ebind_ER_Xenon__100000_thrd1keV 1 / (t yr)']

Er range keep noppBe7:  28 bins from  284.7126501087302  to  2405.

In [13]:
result_pd

,CNO_high_nubb0_allMethod2,CNO_high_nubb0_noppBe7Method2,CNO_low_nubb0_allMethod2,CNO_low_nubb0_noppBe7Method2,pep_high_nubb0_allMethod2,pep_high_nubb0_noppBe7Method2,pep_low_nubb0_allMethod2,pep_low_nubb0_noppBe7Method2
"(0.001, 0.9)",41.614821,20.936206,80.542146,37.438113,152.603688,44.152732,141.917054,38.110073
"(0.01, 0.9)",28.552796,14.268529,55.313346,25.435942,104.58246,30.364276,96.143044,25.821556
"(0.1, 0.9)",14.408829,8.0,27.555957,12.829303,52.098235,15.087134,48.984456,13.179273


In [37]:
result_pd

,CNO_high_nubb0,CNO_low_nubb0,pep_high_nubb0,pep_low_nubb0
"(0.001, 0.9)",42.19277950863805,80.91510434965404,151.7463383197312,141.80703803836363
"(0.01, 0.9)",28.63252280975997,55.53140344080639,104.72855778896678,96.33897722953822
"(0.1, 0.9)",14.393090890907333,27.587930735624113,52.32481364176277,48.87687131067423


In [14]:
file_name =  'CNOpep_Xe_DT_regular_numeNa_Erbinning.csv'
file_name

'CNOpep_Xe_DT_regular_numeNa_Erbinning.csv'

In [15]:
result_pd.to_csv(file_name)